In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

In [ ]:
file_path = '../../data/raw/air-quality/'
files =  os.listdir(file_path)


In [ ]:
files

In [ ]:
# from AirNowAPI documentation
# Outputs: https://docs.airnowapi.org/Data/docs
header_cols = ['lat', 'lon', 'utc', 'parameter','concentration', 'unit', 'raw_concentration',
    'aqi', 'aqi_category', 'site_name', 'site_agency', 'aqs_id', 'full_aqs_id']
for f in files:
    df = pd.read_csv(file_path + f, names=header_cols)
    break

In [ ]:
df.head()